Подготовка данных

In [ ]:
!pip install h5py
!pip install xarray
!pip install h5netcdf

In [ ]:
#OLD_50 
#поставь большой ОЗУ

#https://drive.google.com/file/d/1Db7Np1-AYMhJhXThqFoHpqL0sxJseneU/view?usp=sharing
!gdown --id 1Db7Np1-AYMhJhXThqFoHpqL0sxJseneU

from IPython.display import clear_output

!unzip gaia.zip
!rm -rf 'gaia.zip'


#clear_output()


In [ ]:
!pip uninstall opencv-python-headless 
!pip install opencv-python-headless==4.1.2.30

from IPython.display import clear_output
!pip install -U albumentations
clear_output()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch
from torch.utils.data import Dataset
import glob
from PIL import Image
import numpy as np
import os

from IPython.display import clear_output

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim
import torch.profiler
import torch.utils.data
import torchvision.datasets
import torchvision.models

import albumentations as A

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as I

from osgeo import gdal
import sys

from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim

import os, shutil
import time
from multiprocessing import Pool
from functools import partial
import xarray as xr

#Фиксируем random_seed
import random
import numpy as np
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

#Выставлям device для расчетов
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Создаем класс датасета

In [ ]:
def set_taper(im , coords, shape, taper=0.3):

    window1d = np.abs(np.blackman(shape))
    window2d = np.sqrt(np.outer(window1d,window1d))

    window2d = np.where(window2d>taper, taper, window2d)
    window2d = window2d/np.max(window2d)
    windownew = np.zeros((shape, shape))

    coords = np.round(coords)
    a,b = int(coords[0,1]), int(coords[0,0])
    dx = a-int(shape/2)
    dy = b-int(shape/2)
    x_new = range(0,shape-abs(dx)) if dx<0 else range(dx,shape)
    x_old = range(abs(dx),shape) if dx<0 else range( 0,shape-dx)

    if dy<0:
      windownew[ x_new  , 0:shape-abs(dy)] = window2d[x_old, abs(dy):shape ]
    else:
      windownew[ x_new  ,dy:shape] = window2d[x_old, 0:shape-dy ]

    return  np.multiply(im, windownew)

In [ ]:
import sys
from scipy.ndimage import gaussian_filter


def get_mask(coords , shape, isG, g_sigma, precision=1):
    if precision != 0:
        mask = np.zeros((shape*(precision*10),shape*(precision*10)))
        coords = np.round(coords,precision)*(precision*10)
        coords = np.array( coords, dtype = np.uint8)
        coords = coords[coords[:,0]!=0 ]
        coords = coords[coords[:,1]!=0 ]
        coords = np.where(coords==shape*10, coords-10, coords)
        mask[coords[:,1],coords[:,0]] = 1 
        if isG:
            mask = gaussian_filter(mask, sigma= g_sigma)
            mask /= mask.max()
    else:
        mask = np.zeros((shape,shape))
        coords = np.round(coords)
        coords = np.array( coords, dtype = np.uint8)
        coords = coords[coords[:,0]!=0 ]
        coords = coords[coords[:,1]!=0 ]
        coords = np.where(coords==shape, coords-1, coords)
        mask[coords[:,1],coords[:,0]] = 1 
        if isG:
            mask = gaussian_filter(mask, sigma= g_sigma)
            mask /= mask.max()
    return mask


In [ ]:
class StarDataset(Dataset):
  def __init__(self, path_png, isTaper=False, taper=0.3, isGauss = True,
               gauss=0.75, prec = 1, transform = None, dset=0):
    self.isTaper = isTaper
    self.taper = taper
    self.isGauss = isGauss
    self.gauss = gauss
    self.prec = prec
    assert self.prec >=0, "Set correct precision!"
    self.files_png = path_png
    self.transform = transform
    
    
    self.coords = np.asarray(xr.open_dataarray((self.files_png).replace('images','coords'), engine="h5netcdf") , dtype = np.float32)
    self.len = len(self.coords)
    if dset==1:
      self.coords = self.coords[:int(self.len/2)]  
    if dset==2:
      self.coords = self.coords[int(self.len/2):]  
    self.im = np.asarray(xr.open_dataarray(self.files_png, engine="h5netcdf") , dtype = np.uint8)
    if dset==1:
      self.im = self.im[:int(self.len/2)]  
    if dset==2:
      self.im = self.im[int(self.len/2):]  

    print(self.im.shape)
    print(self.coords.shape)
    index = []
    for i in range(len(self.coords)):
      if self.coords[i, 1, 0] !=0 and self.coords[i, 1, 1] !=0:
          index.append(i)
    self.coords = np.delete(self.coords, index, axis=0)
    self.im = np.delete(self.im, index, axis=0)
    self.len = len(self.coords)
  
  def check(self):
    for f in self.files_png:
      try:
        self.get_coords(f)
      except Exception:
        print(f"File with coords not found for {f}")


  def get_coords(self,f):
    coords_file = f.replace('.png','.txt')
    coords_file = coords_file.replace('image/','coor/')
    coords = np.loadtxt(coords_file)
    return coords
  

  def __getitem__(self, i):

    im = self.im[i]
    coords = self.coords[i]

    if self.transform:
      transformed = self.transform(image = im, keypoints = coords) 
      im = transformed['image']
      im = np.moveaxis(im, -1, 0)
      im = torch.tensor(im, dtype=torch.float32)
      coords =  transformed['keypoints'] 
      shape = im.T.shape
      if self.isTaper:
        im = set_taper(im, coords, shape[0], taper = self.taper)
      coords  = get_mask(coords , shape[0], self.isGauss, self.gauss, self.prec)
      
    return im, coords

  def __len__(self):
    return self.len



Загрузка данных

In [ ]:

def show_img(img, norm=False, figsize=(4,4)):
    plt.figure(figsize=figsize)
    if norm:
        img = img / 2 + 0.5 
    
    npimg = img.numpy()
    if norm:
        npimg = np.clip(npimg, 0., 1.)
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
crop_size = 16
transform_train = A.Compose([
     A.CenterCrop(22, 22),
     A.RandomCrop(crop_size, crop_size),
     A.Flip(p=0.5),
     A.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
], keypoint_params=A.KeypointParams(format='xy'))

transform_val = A.Compose([
     A.CenterCrop(crop_size, crop_size),
     A.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
], keypoint_params=A.KeypointParams(format='xy'))


In [ ]:
train_set = StarDataset("train_images_da50.nc" , 
                        isTaper = False, taper = 0.2, 
                        isGauss = True, gauss = 0.75, 
                        prec = 1, 
                        transform=transform_train)
val_set = StarDataset("val_images_da50.nc" , 
                      isTaper = False, taper = 0.2, 
                      isGauss = True, gauss = 0.75, 
                      prec = 1, 
                      transform=transform_val)
test_set = StarDataset("test_images_da50.nc" , 
                      isTaper = False, taper = 0.2, 
                      isGauss = True, gauss = 0.75, 
                      prec = 1, 
                      transform=transform_val)

In [ ]:
print(len(train_set))
print(len(val_set))
print(len(test_set))

In [ ]:
b_s =128
train_loader = DataLoader(train_set, batch_size=b_s, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=b_s, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_set, batch_size = b_s, shuffle = False, num_workers =4, pin_memory = True)

torch.backends.cudnn.benchmark = True

In [ ]:
%%prun -l 0.1
for img, target in train_loader:
        print(f'Img size {img.size()}')
        print(f'Target size {target.size()}') 
        break

In [ ]:
show_img(torchvision.utils.make_grid(img[:24]), norm=True, figsize=(24,24))

In [ ]:
def coordinates_heatmap(data):
     heatmap, xedges, yedges = np.histogram2d(data.coords[:, :, 0], data.coords[:, :, 0], bins=100)
     extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
     return heatmap, extent

In [ ]:
indx = 5

fig, ax = plt.subplots(ncols=2, figsize=(10,10))
ax[0].imshow((img[indx].permute(1,2,0))/ 2 + 0.5)
ax[1].imshow(target[indx], vmin=0, vmax=1, cmap="Greys")
y,x = torch.where(target[indx] == 1)
print(y,x)
if train_set.prec != 0:
    ax[0].scatter(x/10,y/10, facecolors='white', edgecolors='r', s=5)
else:
    ax[0].scatter(x,y, facecolors='white', edgecolors='r', s=5)

## Модель

In [ ]:
#defining the cnn architecture

class Net(nn.Module):
  def __init__(self, img_size=160):
        super(Net, self).__init__()
       
        #defining maxpool block
        self.maxpool = nn.MaxPool2d(2, 2)
        
        #defining dropout block
        self.dropout = nn.Dropout(p=0.2)
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.bn1 = nn.BatchNorm2d(64)

        #defining second convolutional layer
        self.conv2 = nn.Conv2d(64, 128, 3, bias=False)
        self.bn2 = nn.BatchNorm2d(128)

        
        #defining linear output layer
        self.fc1 = nn.Linear(3200, img_size**2)
        
  def forward(self, x):

        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.bn1(x)

        x = F.relu(self.conv2(x))
        x = self.bn2(x)
        
        x = x.view(x.size(0), -1)


        #passing x through linear layer
        x = self.fc1(x)
        x = torch.sigmoid(x)

        #returning x
        return x

# Обучение

In [ ]:
!pip install torch_tb_profiler
clear_output()

In [ ]:
import shutil
import os
# Helper method to run Tensorboard in Colab
def reinit_tensorboard(clear_log = True, logs = "/content/runs"):
  logs_base_dir = logs # Directory for log files
  if clear_log:    
    shutil.rmtree(logs_base_dir, ignore_errors = True)
    os.makedirs(logs_base_dir, exist_ok=True)
  # Colab magic
  %reload_ext tensorboard
  %tensorboard --logdir {logs_base_dir}

In [ ]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice


In [ ]:
import numpy as np
import copy

def get_std(image):
    return image.std()

def get_max(image,alpha=20,size=10, prec=10):
    size *= prec
    sigma = get_std(image)
    i_out = []
    j_out = []
    image_temp = copy.deepcopy(image)
    while True:
        k = np.argmax(image_temp)
        j,i = np.unravel_index(k, image_temp.shape)
        if(image_temp[j,i] >= sigma*alpha):
            i_out.append(i)
            j_out.append(j)
            x = np.arange(i-size, i+size)
            y = np.arange(j-size, j+size)
            xv,yv = np.meshgrid(x,y)
            image_temp[yv.clip(0,image_temp.shape[0]-1),
                                   xv.clip(0,image_temp.shape[1]-1) ] = 0
        else:
            break
    return i_out,j_out

def get_range(a, b):
   dx = (a[0]-b[0])
   dy = (a[1]-b[1])
   return (dx*dx+dy*dy)**0.5

def get_mean(pred, targ, prec):
    mean =0. 
    k=0
    for i in range(len(pred)):
      min=1e9
      for j in range(len(targ)):
        dr = get_range(pred[i], targ[j])
        if min>dr:
          min=dr
      if min<4*prec: 
        k+=1
        mean+=min
    return mean, k

In [ ]:
from PIL import Image

indx = 0

def argmax2d(X):
    n, m = X.shape
    x_ = torch.ravel(X)
    k = torch.argmax(x_)
    i, j = k // m, k % m
    return i, j

def show(model, dataloader, indx=0, thres=0.2, prec = 10):
  model.eval()
  for image, target in dataloader:
    break  
  correct = 0
  total=0
  with torch.no_grad():
    image = image.type(torch.FloatTensor)
    image = torch.unsqueeze(image[indx], 0)
    pred = model(image.to(device))
    pred = pred.cpu()
    pred = torch.reshape(pred, (-1,  target.size(2),  target.size(2)))
    pred_i_out, pred_j_out = get_max(pred[indx],alpha=0.5,size=3, prec=10)
    c = np.vstack((pred_i_out, pred_j_out)).T
    targ_i_out, targ_j_out = get_max(target[indx],alpha=0.5,size=3, prec=10)
    d = np.vstack((targ_i_out, targ_j_out)).T
    correct += np.sum(np.array_equal(c,d))
    total += len(d)
    dr, num = get_mean(c,d, prec)
    if num!=0:
      dr = dr/num/prec
    else:
      dr=-1

  fig,ax = plt.subplots(ncols=3)
  ax[0].imshow((image[indx]/ 2 + 0.5).permute(1,2,0), vmin=0, vmax=1)
  ax[1].imshow(torch.sum((image[indx]/ 2 + 0.5).permute(1,2,0),  dim=-1)/3, vmin=0, vmax=1)
  y_target,x_target = torch.where(target[indx] == 1)
  
  ax[2].imshow(pred[indx],cmap='inferno', vmin=0, vmax=1)
  y_pred,x_pred = argmax2d(pred[indx])
  
  ax[0].scatter(x_target/prec,y_target/prec, facecolors='green', edgecolors='white', alpha=0.75)
  ax[0].scatter(x_pred/prec,y_pred/prec, facecolors='red', edgecolors='white', alpha=0.75)

  ax[1].scatter(x_pred/prec,y_pred/prec, facecolors='red', edgecolors='white', alpha=0.75)
  ax[1].scatter(x_target/prec,y_target/prec, facecolors='green', edgecolors='white', alpha=0.75)

  ax[2].scatter(x_pred,y_pred, facecolors='red', edgecolors='white', alpha=0.75)
  ax[2].scatter(x_target,y_target, facecolors='green', edgecolors='white', alpha=0.75)


  ax[0].set_title('Input')
  ax[1].set_title('Brightness')
  ax[2].set_title('Predicted mask')
  fig.suptitle('dr= ' + str(round(dr,2)) + " pix | correct/total= " + str(correct) + "/" + str(total))
  return fig

In [ ]:
import copy

def validate(model, dataloader, device, part, prec=10):
    l = len(dataloader)*part*b_s
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    mean = 0
    total_mean = 0
    with torch.no_grad():
        for images, targets in dataloader:
          images = images.type(torch.FloatTensor)
          pred = model(images.to(device))
          pred = pred.to('cpu')
          pred = torch.reshape(pred, (-1,  targets.size(2),  targets.size(2)))
          for i in range((pred.size(0))):
              pred_i_out, pred_j_out = get_max(pred[i],alpha=0.5,size=3, prec=10)
              c = np.vstack((pred_i_out, pred_j_out)).T
              targ_i_out, targ_j_out = get_max(targets[i],alpha=0.5,size=3, prec=10)
              d = np.vstack((targ_i_out, targ_j_out)).T
              correct += np.sum(np.array_equal(c,d))
              total += len(d)
              dr, num = get_mean(c,d, prec)
              mean +=dr
              total_mean+=num
              if total > l: 
                  break
          if total > l: 
              break
    print('Correct stars: %i out of %i' % (correct, total))
    if total_mean!=0:
      dr = mean/total_mean/prec
    else:
      dr=-1
    return correct/total, dr

In [ ]:
def train(model, comment = "", epoch_n = 10, prec = 10):
  writer = SummaryWriter(comment)
  # Run model on cuda
  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  print('Using device:', device)
  torch.cuda.empty_cache()
  model.train()
  model.to(device)

  criterion = DiceLoss()
  optimizer = optim.Adam(model.parameters(), lr = 3e-4)
  best_accuracy = 0
  n_epochs = epoch_n
  step=0
  best_acc = 1e9
  for epoch in range(n_epochs):
    running_loss = 0.0
    model.train()
    for batch_i, data in enumerate(train_loader):
            # get the input images and their corresponding labels
            images = data[0]
            key_pts = data[1]
            # flatten pts
            key_pts = key_pts.view(key_pts.size(0), -1)

            key_pts = key_pts.type(torch.FloatTensor)
            images = images.type(torch.FloatTensor)
            key_pts, images = key_pts.to(device), images.to(device) #Чаще делают вот так
           
            # forward pass to get outputs
            output_pts = model(images)
            # calculate the loss between predicted and target keypoints
            loss = criterion(output_pts, key_pts)
            
            # zero the parameter (weight) gradients
            optimizer.zero_grad(set_to_none=True)
            # backward pass to calculate the weight gradients
            loss.backward()
            # update the weights
            optimizer.step()
            # print loss statistics
            running_loss += loss.item()
            if batch_i % 100 == 0:    # print every 10 batches
                print('Epoch: {}, Batch: {}/{}, Loss: {}'.format(epoch, batch_i+1, len(train_loader), loss.item()))
                running_loss = 0.0
                writer.add_scalar('Loss/train_step',loss.item(),step)
                writer.flush()
            step+=1
    writer.add_scalar('Loss/train_epoch',loss.item(),epoch)
    
    model.eval()
    accuracy_train , error_train = validate(model,train_loader,device, 0.0625, prec =prec )
    writer.add_scalar('Accuracy/Accuracy_train',accuracy_train,epoch)
    writer.add_scalar('Error/Error_train',error_train,epoch)

    accuracy_val ,error_val = validate(model,val_loader,device, 1, prec =prec )
    writer.add_scalar('Accuracy/Accuracy_val',accuracy_val,epoch)
    writer.add_scalar('Error/Error_val',error_val,epoch)

    writer.add_figure("Images/Images_train", show(model, train_loader, prec =prec ), epoch)
    writer.add_figure("Images/Images_validate", show(model, val_loader, prec =prec ), epoch)
     
    print("Epoch {} Loss {:.5f} Error_train {:.5f} Error_validate {:.5f} \nAccuracy_train {:.5f} Accuracy_validate {:.5f}".format(epoch, loss.item(),  error_train, error_val, accuracy_train, accuracy_val))
    writer.flush()

    
    if error_val <best_acc: 
      best_acc = error_val
      namem = "Gaia" +str(round(error_val,4))
      torch.save(net, "drive/MyDrive/Star_SuperRes/" + namem)
            
  writer.close()

In [ ]:
reinit_tensorboard(clear_log=True) # If needed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#net = torch.load('drive/MyDrive/Star_SuperRes/Gaia0.3184')

net = torch.load('drive/MyDrive/Star_SuperRes/Last_Mask_old50_0.5761')
#torch.save(net, "Model_start")


In [ ]:
import gc
gc.collect()
prec=10
net = Net(img_size=16*prec)
torch.cuda.empty_cache()
train(net, epoch_n=1000, prec =prec)

# Тестируем

In [ ]:

def get_range(a, b):
   dx = (a[0]-b[0])
   dy = (a[1]-b[1])
   return (dx*dx+dy*dy)**0.5



def get_mean(pred, targ, prec):
    mean =0. 
    k=0
    dx, dy = 0,0
    x, y = 0,0
    for i in range(len(targ)):
      min=1e9
      for j in range(len(pred)):
        dr = get_range(targ[i], pred[j])
        if min>dr:
          min=dr
          dx= abs(targ[i, 0]- pred[j, 0])
          dy= abs(targ[i, 1]- pred[j, 1])
      if min<4*prec: 
        k+=1
        mean+=min
        x, y = dx,dy

    return mean, x, y, k


def validate(model, dataloader, device, part, prec=10):
    l = len(dataloader)*part*b_s
    lr=[]
    lx=[]
    ly=[]
    model.to(device)
    model.eval()
    correct = 0
    total = 0
    mean = 0
    meanx = 0
    meany = 0
    total_mean = 0
    with torch.no_grad():
        for images, targets in dataloader:
          images = images.type(torch.FloatTensor)
          pred = model(images.to(device))
          pred = pred.to('cpu')
          pred = torch.reshape(pred, (-1,  targets.size(2),  targets.size(2)))
          for i in range((pred.size(0))):
              pred_i_out, pred_j_out = get_max(pred[i],alpha=0.5,size=3, prec=prec)
              c = np.vstack((pred_i_out, pred_j_out)).T
              targ_i_out, targ_j_out = get_max(targets[i],alpha=0.5,size=3, prec=prec)
              d = np.vstack((targ_i_out, targ_j_out)).T
              correct += np.sum(np.array_equal(c,d))
              total += len(d)
              dr, dx, dy, num = get_mean(c,d, prec)
              lr.append(dr)
              lx.append(dx)
              ly.append(dy)
              mean +=dr
              meanx +=dx
              meany +=dy
              total_mean+=num
              if total > l: 
                  break
          if total > l: 
              break
    print('Correct stars: %i out of %i' % (correct, total))
    dr = mean/total_mean/prec
    dx = meanx/total_mean/prec
    dy = meany/total_mean/prec

    f = open( 'dr.txt', 'w' )
    for item in lr:
      f.write("%s\n" % str(item))
    f.close()

    f = open( 'dx.txt', 'w' )
    for item in lx:
      f.write("%s\n" % str(item))
    f.close()

    f = open( 'dy.txt', 'w' )
    for item in ly:
      f.write("%s\n" % str(item))
    f.close()

    return correct/total, dr, dx, dy



def val_std(model, dataloader, device, part, mean_dr ,mean_dx ,mean_dy ,prec=10):
    l = len(dataloader)*part*b_s
    model.to(device)
    model.eval()
    mean = 0
    meanx = 0
    meany = 0
    total=0
    total_mean = 0
    with torch.no_grad():
        for images, targets in dataloader:
          images = images.type(torch.FloatTensor)
          pred = model(images.to(device))
          pred = pred.to('cpu')
          pred = torch.reshape(pred, (-1,  targets.size(2),  targets.size(2)))
          for i in range((pred.size(0))):
              pred_i_out, pred_j_out = get_max(pred[i],alpha=0.5,size=3, prec=prec)
              c = np.vstack((pred_i_out, pred_j_out)).T
              targ_i_out, targ_j_out = get_max(targets[i],alpha=0.5,size=3, prec=prec)
              d = np.vstack((targ_i_out, targ_j_out)).T
              total += len(d)
              if prec!=10: print("1pix")
              dr, dx, dy, num = get_mean(c,d, prec)
              sum= (dr/prec-mean_dr)*(dr/prec-mean_dr)
              mean +=sum
              sumx= (dx/prec-mean_dx)*(dx/prec-mean_dx)
              meanx +=sumx
              sumy= (dy/prec-mean_dy)*(dy/prec-mean_dy)
              meany +=sumy
              total_mean+=num
              if total > l: 
                  break
          if total > l: 
              break
    dr = mean/total_mean
    dx = meanx/total_mean
    dy = meany/total_mean
    return (dr)**0.5, (dx)**0.5, (dy)**0.5, total_mean


In [ ]:
net = torch.load('drive/MyDrive/Star_SuperRes/Last_Mask_old50_0.5761')
#net = torch.load('drive/MyDrive/Star_SuperRes/Last_Mask_old50_1pix0.5683')

In [ ]:
prec =10
test_set.prec=1
accuracy_test , error_test , error_x, error_y= validate(net,test_loader,device, 1, prec =prec)
print("Accuracy ", round(accuracy_test*100, 2), "%")

std_test , std_x, std_y, total= val_std(net,test_loader,device, 1, error_test,error_x, error_y, 
                                 prec =prec )
st = 3.291
eps_test, eps_x, eps_y = std_test/(total-1)*st , std_x/(total-1)*st, std_y/(total-1)*st
print()
print("Error ", round(error_test, 2), "pix")
print("Eps ", round(eps_test, 2), "pix")
print("STD ", round(std_test, 2), "pix")
print()
print("ErrorX ", round(error_x, 2), "pix")
print("Eps ", round(eps_x, 2), "pix")
print("STDX ", round(std_x, 2), "pix")
print()
print("ErrorY ", round(error_y, 2), "pix")
print("Eps ", round(eps_y, 2), "pix")
print("STDY ", round(std_y, 2), "pix")
